In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import pandas as pd
from tqdm.notebook import tqdm
#from time import sleep

In [2]:
class HaarFeature:
    def __init__(self,tipe,r,c,h,w):
        self.tipe = tipe
        self.r = r
        self.c = c
        self.h = h
        self.w = w

In [3]:
class Classifier:
    score = 0
    weight = 0
    polarity = 0
    threshold = 0
    def __init__(self,haarFeature):
        self.haarFeature = haarFeature

In [4]:
def generateIntegralImage(img):
    img_h = img.shape[0]
    img_w = img.shape[1]
    img_integral = np.zeros(img.shape,dtype='int64')
    for row in range(img_h):
        for col in range(img_w):
            a = img_integral[row-1,col-1] if row-1>=0 and col-1>=0 else 0
            b = img_integral[row-1,col] if row-1>=0 else 0
            c = img_integral[row,col-1] if col-1>=0 else 0
            d = img[row,col]
            img_integral[row,col] = d+b+c-a
    return img_integral

In [5]:
def generateDeltaValue(img_integral,f):
    delta = 0
    feature_h = f.h
    feature_w = f.w
    feature_r = f.r
    feature_c = f.c
    if f.tipe==1:
        b_h = feature_h
        b_w = int(feature_w/2)
        b_r = feature_r
        b_c = feature_c
        
        a = 0 if feature_r-1<0 or feature_c-1<0 else img_integral[feature_r-1,feature_c-1]
        b = 0 if feature_r-1<0 or feature_c+feature_w-1<0 else img_integral[feature_r-1,feature_c+feature_w-1]
        c = 0 if feature_r+feature_h-1<0 or feature_c-1<0 else img_integral[feature_r+feature_h-1,feature_c-1]
        d = img_integral[feature_r+feature_h-1,feature_c+feature_w-1]
        e = int(d)-int(b)-int(c)+int(a)
        #print(a,' ',b,' ',c,' ',d)
        #print(f.r,' ',f.c,' ',f.h,' ',f.w, 'e:',e)
        
        a2 = 0 if b_r-1<0 or b_c-1<0 else img_integral[b_r-1,b_c-1]
        b2 = 0 if b_r-1<0 or b_c+b_w-1<0 else img_integral[b_r-1,b_c+b_w-1]
        c2 = 0 if b_r+b_h-1<0 or b_c-1<0 else img_integral[b_r+b_h-1,b_c-1]
        d2 = img_integral[b_r+b_h-1,b_c+b_w-1]
        e2 = int(d2)-int(c2)-int(b2)+int(a2)
        #print(b_r,' ',b_c,' ',b_h,' ',b_w, 'e2:',e2)
        #print(a2,' ',b2,' ',c2,' ',d2)
        
        delta = e - e2 - e2
        #print(classifier.score,':',a,' ',b,' ',c,' ',d)
    elif f.tipe==2:
        b_h = int(feature_h/2)
        b_w = feature_w
        b_r = feature_r+int(feature_h/2)
        b_c = feature_c
        
        a = 0 if feature_r-1<0 or feature_c-1<0 else img_integral[feature_r-1,feature_c-1]
        b = 0 if feature_r-1<0 or feature_c+feature_w-1<0 else img_integral[feature_r-1,feature_c+feature_w-1]
        c = 0 if feature_r+feature_h-1<0 or feature_c-1<0 else img_integral[feature_r+feature_h-1,feature_c-1]
        d = img_integral[feature_r+feature_h-1,feature_c+feature_w-1]
        e = int(d)-int(b)-int(c)+int(a)
        
        a2 = 0 if b_r-1<0 or b_c-1<0 else img_integral[b_r-1,b_c-1]
        b2 = 0 if b_r-1<0 or b_c+b_w-1<0 else img_integral[b_r-1,b_c+b_w-1]
        c2 = 0 if b_r+b_h-1<0 or b_c-1<0 else img_integral[b_r+b_h-1,b_c-1]
        d2 = img_integral[b_r+b_h-1,b_c+b_w-1]
        e2 = int(d2)-int(c2)-int(b2)+int(a2)
        
        delta = e - e2 - e2
    elif f.tipe==3:
        b_h = int(feature_h/3)
        b_w = feature_w
        b_r = feature_r+int(feature_h/3)
        b_c = feature_c
        
        a = 0 if feature_r-1<0 or feature_c-1<0 else img_integral[feature_r-1,feature_c-1]
        b = 0 if feature_r-1<0 or feature_c+feature_w-1<0 else img_integral[feature_r-1,feature_c+feature_w-1]
        c = 0 if feature_r+feature_h-1<0 or feature_c-1<0 else img_integral[feature_r+feature_h-1,feature_c-1]
        d = img_integral[feature_r+feature_h-1,feature_c+feature_w-1]
        e = int(d)-int(b)-int(c)+int(a)
        
        a2 = 0 if b_r-1<0 or b_c-1<0 else img_integral[b_r-1,b_c-1]
        b2 = 0 if b_r-1<0 or b_c+b_w-1<0 else img_integral[b_r-1,b_c+b_w-1]
        c2 = 0 if b_r+b_h-1<0 or b_c-1<0 else img_integral[b_r+b_h-1,b_c-1]
        d2 = img_integral[b_r+b_h-1,b_c+b_w-1]
        e2 = int(d2)-int(c2)-int(b2)+int(a2)
        
        delta = 2*e2-e
        
    elif f.tipe==4:
        b_h = feature_h
        b_w = int(feature_w/3)
        b_r = feature_r
        b_c = feature_c+int(feature_w/3)
        
        a = 0 if feature_r-1<0 or feature_c-1<0 else img_integral[feature_r-1,feature_c-1]
        b = 0 if feature_r-1<0 or feature_c+feature_w-1<0 else img_integral[feature_r-1,feature_c+feature_w-1]
        c = 0 if feature_r+feature_h-1<0 or feature_c-1<0 else img_integral[feature_r+feature_h-1,feature_c-1]
        d = img_integral[feature_r+feature_h-1,feature_c+feature_w-1]
        e = int(d)-int(b)-int(c)+int(a)
        
        a2 = 0 if b_r-1<0 or b_c-1<0 else img_integral[b_r-1,b_c-1]
        b2 = 0 if b_r-1<0 or b_c+b_w-1<0 else img_integral[b_r-1,b_c+b_w-1]
        c2 = 0 if b_r+b_h-1<0 or b_c-1<0 else img_integral[b_r+b_h-1,b_c-1]
        d2 = img_integral[b_r+b_h-1,b_c+b_w-1]
        e2 = int(d2)-int(c2)-int(b2)+int(a2)
        
        delta = 2*e2-e
    
    elif f.tipe==5:
        a = 0 if feature_r-1<0 or feature_c-1<0 else img_integral[feature_r-1,feature_c-1]
        b = 0 if feature_r-1<0 or feature_c+feature_w-1<0 else img_integral[feature_r-1,feature_c+feature_w-1]
        c = 0 if feature_r+feature_h-1<0 or feature_c-1<0 else img_integral[feature_r+feature_h-1,feature_c-1]
        d = img_integral[feature_r+feature_h-1,feature_c+feature_w-1]
        e = int(d)-int(b)-int(c)+int(a)
        
        b_h = int(feature_h/2)
        b_w = int(feature_w/2)
        b_r = feature_r 
        b_c = feature_c
        c_r = feature_r + b_h
        c_c = feature_c + b_w
        
        a2 = 0 if b_r-1<0 or b_c-1<0 else img_integral[b_r-1,b_c-1]
        b2 = 0 if b_r-1<0 or b_c+b_w-1<0 else img_integral[b_r-1,b_c+b_w-1]
        c2 = 0 if b_r+b_h-1<0 or b_c-1<0 else img_integral[b_r+b_h-1,b_c-1]
        d2 = img_integral[b_r+b_h-1,b_c+b_w-1]
        e2 = int(d2)-int(b2)-int(c2)+int(a2)
        
        a3 = 0 if c_r-1<0 or c_c-1<0 else img_integral[c_r-1,c_c-1]
        b3 = 0 if c_r-1<0 or c_c+b_w-1<0 else img_integral[c_r-1,c_c+b_w-1]
        c3 = 0 if c_r+b_h-1<0 or c_c-1<0 else img_integral[c_r+b_h-1,c_c-1]
        d3 = img_integral[c_r+b_h-1,c_c+b_w-1]
        e3 = int(d3)-int(b3)-int(c3)+int(a3)
        
        delta = (e-e2-e3)-e2-e3
    return delta

In [6]:
training_faces=[]
training_nonfaces=[]
#test_faces=[]
#test_nonfaces=[]
for img in glob.glob('images2/faces/*.pgm'):
    img = cv2.imread(img,0)
    training_faces.append(img)
for img in glob.glob('images2/non-faces/*.pgm'):
    img = cv2.imread(img,0)
    training_nonfaces.append(img)

img = cv2.imread('img/face00001.png',0)
plt.imshow(img,cmap='gray')

img.shape

In [7]:
#generate haar feature
features = []
img = training_faces[0]
im_height = img.shape[0]
im_width = img.shape[1]
tipe = 1
h = 1
w = 2
for var_h in range(h,im_height+1,h):
    for var_w in range(w,im_width+1,w):
        for var_r in range(0,im_height-var_h+1):
            for var_c in range(0,im_width-var_w+1):
                haarFeature = HaarFeature(tipe,var_r,var_c,var_h,var_w)
                features.append(haarFeature)

tipe = 2
h = 2
w = 1
for var_h in range(h,im_height+1,h):
    for var_w in range(w,im_width+1,w):
        for var_r in range(0,im_height-var_h+1):
            for var_c in range(0,im_width-var_w+1):
                haarFeature = HaarFeature(tipe,var_r,var_c,var_h,var_w)
                features.append(haarFeature)
                
tipe = 3
h = 3
w = 1
for var_h in range(h,im_height+1,h):
    for var_w in range(w,im_width+1,w):
        for var_r in range(0,im_height-var_h+1):
            for var_c in range(0,im_width-var_w+1):
                haarFeature = HaarFeature(tipe,var_r,var_c,var_h,var_w)
                features.append(haarFeature)
                
tipe = 4
h = 1
w = 3
for var_h in range(h,im_height+1,h):
    for var_w in range(w,im_width+1,w):
        for var_r in range(0,im_height-var_h+1):
            for var_c in range(0,im_width-var_w+1):
                haarFeature = HaarFeature(tipe,var_r,var_c,var_h,var_w)
                features.append(haarFeature)

tipe = 5
h = 2
w = 2
for var_h in range(h,im_height+1,h):
    for var_w in range(w,im_width+1,w):
        for var_r in range(0,im_height-var_h+1):
            for var_c in range(0,im_width-var_w+1):
                haarFeature = HaarFeature(tipe,var_r,var_c,var_h,var_w)
                features.append(haarFeature)

len(features)

In [13]:
training_faces_integral=[]
training_nonfaces_integral=[]
for img in training_faces:
    img_integral = generateIntegralImage(img)
    training_faces_integral.append(img_integral)
for img in training_nonfaces:
    img_integral = generateIntegralImage(img)
    training_nonfaces_integral.append(img_integral)

In [14]:
kolom = ['label','weight']
#for f in features[0:1]:
for f in features:
    kolom.append("[{tipe}:{r},{c}:{h},{w}]".format(tipe=f.tipe,r=f.r,c=f.c,h=f.h,w=f.w))
df = pd.DataFrame(columns=kolom)
#df.to_csv('dataframe.csv',index=False)

In [15]:
df

,label,weight,"[1:0,0:1,2]","[1:0,1:1,2]","[1:0,2:1,2]","[1:0,3:1,2]","[1:0,4:1,2]","[1:0,5:1,2]","[1:0,6:1,2]","[1:0,7:1,2]",...,"[5:0,2:18,16]","[5:0,3:18,16]","[5:1,0:18,16]","[5:1,1:18,16]","[5:1,2:18,16]","[5:1,3:18,16]","[5:0,0:18,18]","[5:0,1:18,18]","[5:1,0:18,18]","[5:1,1:18,18]"


In [16]:
#t=training_faces_integral[10]
#f= features[0]

#print(t)
#print("[{tipe}:{r},{c}:{h},{w}]".format(tipe=f.tipe,r=f.r,c=f.c,h=f.h,w=f.w))

#print(generateDeltaValue(img_integral,f))

#df.drop(df.index[:],inplace=True)
#data = 1000
n_faces = len(training_faces_integral)
n_nonfaces = len(training_nonfaces_integral)
for i in tqdm(range(n_faces)):
    img_integral = training_faces_integral[i]
    #for index,img_integral in enumerate(training_faces_integral[:]):
    series=[1,0]
    #for f in features[0:1]:
    for f in features:
        deltavalue = generateDeltaValue(img_integral,f)
        series.append(deltavalue)
    df.loc[len(df)] = series
    #if i % 100 == 0:
        #df.to_csv('dataframe.csv',index=False,mode='a',header=False)
        #df.drop(df.index,inplace=True)
#df.to_csv('dataframe.csv',index=False,mode='a',header=False)
#df.drop(df.index,inplace=True)

for i in tqdm(range(n_nonfaces)):
    img_integral = training_nonfaces_integral[i]
    #for img_integral in training_nonfaces_integral[:]:
    series=[0,0]
    #for f in features[0:1]:
    for f in features:
        deltavalue = generateDeltaValue(img_integral,f)
        series.append(deltavalue)
    df.loc[len(df)] = series
    #if i % 100 == 0:
        #df.to_csv('dataframe.csv',index=False,mode='a',header=False)
        #df.drop(df.index,inplace=True)
#df.to_csv('dataframe.csv',index=False,mode='a',header=False)
#df.drop(df.index,inplace=True)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

In [17]:
df

,label,weight,"[1:0,0:1,2]","[1:0,1:1,2]","[1:0,2:1,2]","[1:0,3:1,2]","[1:0,4:1,2]","[1:0,5:1,2]","[1:0,6:1,2]","[1:0,7:1,2]",...,"[5:0,2:18,16]","[5:0,3:18,16]","[5:1,0:18,16]","[5:1,1:18,16]","[5:1,2:18,16]","[5:1,3:18,16]","[5:0,0:18,18]","[5:0,1:18,18]","[5:1,0:18,18]","[5:1,1:18,18]"
0,1,0,18,20,17,3,-4,9,10,8,...,-793,-1261,380,242,-603,-854,-183,-685,-16,-452
1,1,0,22,26,12,11,-3,4,20,-7,...,-862,-1481,470,163,-690,-1094,-154,-766,-100,-561
2,1,0,18,23,23,16,7,-1,10,11,...,-316,-1394,264,235,-427,-1285,200,-209,-46,-328
3,1,0,3,0,8,13,-12,6,28,9,...,-95,-1250,211,115,-256,-1282,63,-156,-207,-312
4,1,0,14,27,13,2,0,14,15,10,...,-339,-1183,339,137,-527,-1256,240,-304,-42,-514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0,0,9,40,14,-30,-24,23,0,-12,...,-255,-13,565,819,1033,1231,-430,-219,924,1163
61,0,0,6,-1,-3,1,0,-9,-1,1,...,-317,-319,-628,-480,-424,-424,-509,-510,-668,-623
62,0,0,0,2,48,82,-5,6,-29,-70,...,4998,4238,6455,6305,5611,4768,6434,5973,7439,6807
63,0,0,2,-1,-1,-1,-1,-1,0,0,...,-56,-54,-123,-84,-48,-53,-135,-89,-124,-80


#df.to_csv('dataframe.csv',index=False)

#len(training_faces_integral)

#df.drop(df.index,inplace=True)

df = pd.read_csv('dataframe.csv')
df.loc[212]['w']=1/len(df)

In [22]:
df.sort_values(df.columns[2],inplace=True,ascending=False)
df

,label,weight,"[1:0,0:1,2]","[1:0,1:1,2]","[1:0,2:1,2]","[1:0,3:1,2]","[1:0,4:1,2]","[1:0,5:1,2]","[1:0,6:1,2]","[1:0,7:1,2]",...,"[5:0,2:18,16]","[5:0,3:18,16]","[5:1,0:18,16]","[5:1,1:18,16]","[5:1,2:18,16]","[5:1,3:18,16]","[5:0,0:18,18]","[5:0,1:18,18]","[5:1,0:18,18]","[5:1,1:18,18]"


In [16]:
tp=tn=fp=fn = 0
threshold = -9999999
neg = 9999999
thres_min = 0
for i in range(df.shape[0]):
    buff = df.iloc[i][2]
    if(buff!=threshold):
        threshold = buff
        for j in range(df.shape[0]):
            kelas = df.iloc[j]['label']
            buff2 = df.iloc[j][2]
            if(buff2>=threshold and kelas==1):
                tp = tp+1
            if(buff2>=threshold and kelas==0):
                fp = fp+1
            if(buff2<=threshold and kelas==0):
                tn = tn+1
            if(buff2<=threshold and kelas==1):
                fn = fn+1
        print('thres:',threshold,' tp:',tp,' fp:',fp,' tn:',tn,' fn:',fn)
        if(fp+fn<neg):
            neg = fp+fn
            thres_min = threshold
        tp=tn=fp=fn = 0
print('threshold minimum : ',thres_min)

thres: 29  tp: 1  fp: 0  tn: 45  fn: 20
thres: 27  tp: 1  fp: 1  tn: 45  fn: 19
thres: 26  tp: 1  fp: 2  tn: 44  fn: 19
thres: 24  tp: 1  fp: 4  tn: 43  fn: 19
thres: 23  tp: 2  fp: 4  tn: 41  fn: 19
thres: 22  tp: 3  fp: 4  tn: 41  fn: 18
thres: 20  tp: 4  fp: 4  tn: 41  fn: 17
thres: 18  tp: 6  fp: 4  tn: 41  fn: 16
thres: 17  tp: 7  fp: 4  tn: 41  fn: 14
thres: 15  tp: 7  fp: 5  tn: 41  fn: 13
thres: 14  tp: 8  fp: 6  tn: 40  fn: 13
thres: 13  tp: 9  fp: 6  tn: 39  fn: 12
thres: 12  tp: 11  fp: 6  tn: 39  fn: 11
thres: 11  tp: 12  fp: 6  tn: 39  fn: 9
thres: 9  tp: 13  fp: 8  tn: 39  fn: 8
thres: 8  tp: 13  fp: 9  tn: 37  fn: 7
thres: 6  tp: 14  fp: 11  tn: 36  fn: 7
thres: 5  tp: 15  fp: 13  tn: 34  fn: 6
thres: 3  tp: 16  fp: 14  tn: 32  fn: 5
thres: 2  tp: 17  fp: 18  tn: 31  fn: 4
thres: 1  tp: 18  fp: 20  tn: 27  fn: 3
thres: 0  tp: 18  fp: 29  tn: 25  fn: 2
thres: -1  tp: 18  fp: 31  tn: 16  fn: 2
thres: -2  tp: 18  fp: 32  tn: 14  fn: 2
thres: -3  tp: 18  fp: 36  tn: 13  fn: 